# Predicting Entities<br>Adam Klie<br>11/02/2019<br>Script to predict entities in from trained model

#### Import necessary packages

In [1]:
# Data processing
import numpy as np
import pandas as pd
from sklearn import preprocessing

# Data visualization
from tqdm import tqdm
import matplotlib
import seaborn as sns

# NLP
import re
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from string import punctuation

# Neural nets
from keras.models import load_model

/cellar/users/aklie/miniconda3/envs/deep_nlp_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/cellar/users/aklie/miniconda3/envs/deep_nlp_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/cellar/users/aklie/miniconda3/envs/deep_nlp_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/cellar/users/aklie/miniconda3/envs/deep_nlp_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
# Function to embed tokens from text into word embedding space
def get_features(docs, max_length):
    docs = list(docs)
    Xs = np.zeros((len(docs), max_length), dtype='int32')
    for i, doc in enumerate(docs):
        j = 0
        for token in doc:
            vector_id = token.vocab.vectors.find(key=token.orth)
            if vector_id >= 0:
                Xs[i, j] = vector_id
            else:
                Xs[i, j] = 0
            j += 1
            if j >= max_length:
                break
    return Xs

#### Import trained model, word embedding, and data to build validation data of off

In [3]:
model_iter = 'simple'

In [4]:
nlp = spacy.load('../data/wikipedia-pubmed-and-PMC-w2v')

/cellar/users/aklie/miniconda3/envs/deep_nlp_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/cellar/users/aklie/miniconda3/envs/deep_nlp_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [ ]:
le = preprocessing.LabelEncoder()
le.classes_ = np.load('../results/{model}_classes.npy'.format(model = model_iter))
model = load_model('../models/{model}.h5'.format(model = model_iter))

In [ ]:
model.summary()

In [ ]:
pd.DataFrame(model.predict_proba(get_features(nlp.pipe(["BRAF mutation"]), 7)), columns=le.classes_)

In [ ]:
SRS_dir = "../data/allSRS_05_15_2018.pickle"
allSRS = pd.read_pickle(SRS_dir)

In [ ]:
# Random titles as validation data
#random_data = allSRS[allSRS.index.get_level_values(1) == 'TITLE'].sample(n = 500)

In [ ]:
# Read in validation data for a specific class to predict on
validation_class = 'Tissue'
filename = '../results/{myclass}_validation_set.pickle'.format(model = model_iter, myclass = validation_class)
readin_data = pd.read_pickle(filename)

In [ ]:
validation_data = readin_data
validation_data.shape[0]

#### Breaking up into sentences

In [ ]:
processed_test = validation_data.str.split('[;.,]', expand = True).stack()
processed_test = processed_test.str.replace('\s+', ' ')

In [ ]:
processed_test.head()

In [ ]:
# Predict the empty state to use as baseline probability emission
val_docs = list(nlp.pipe(' '))
val_X = get_features(val_docs, max_length = model.input_shape[1])
emptyState = model.predict_proba(val_X)[0,:]
emptyState

In [ ]:
%%time
stopWords = set(stopwords.words('english'))
rows = []
key_list = []
for i, (key, sent) in enumerate(tqdm(processed_test.items(), total=len(processed_test))):
    
    # Sentence preprocessing
    #sent = re.sub(r'[^a-zA-Z0-9]+', ' ', sent)  # remove non alpha numeric characters
    tokens = re.split(pattern = ' ', string = sent)  # tokenize the description
    tokens = list(filter(lambda token:(token!='') and (token not in stopWords), tokens))  # filter out stopwords
    sent = ' '.join(tokens)
    
    n_gram_max = min([len(tokens), 7])
    for n_gram in range(2, n_gram_max + 1):
        
        # Get prediction for all current n-grams
        grams = list(map(lambda L:" ".join(L), list(ngrams(tokens, n_gram))))  
        val_docs = list(nlp.pipe(grams))  # get spacy objects for each token passed in
        val_X = get_features(val_docs, max_length = model.input_shape[1])
        predictM = model.predict_proba(val_X)
        
        # Take only those n-grams that have a total probability greater than the empty state + 0.01
        # and also have two tokens present in word-embedding
        tmp_df = pd.DataFrame(data = predictM, columns = le.classes_, index = grams)
        empty_mask = (tmp_df - emptyState).abs().sum(axis=1) < 0.01
        moreThanTwoValToken_mask = (val_X != 0).sum(axis=1) >= 2
        tmp_df[empty_mask&moreThanTwoValToken_mask] = 0
        
        # Set up keys for dataframe with probabilities of each n-gram, will be useful later
        for j, gram in enumerate(tmp_df.index):
            i_end = j + n_gram
            textBefore = " ".join(tokens[:j]) + ('' if j==0 else ' ')
            start_char_pos = len(textBefore)
            key_list.append(key + (i, sent, n_gram, j, i_end, gram, start_char_pos)) 
            rows.append(tmp_df.iloc[j])

In [ ]:
proba_df = pd.concat(rows, keys = key_list, axis = 1).T
proba_df.index.names = ['srs', 'attribute', 'sentence_number', 'kthSrs', 
                        'orig_text', 'n-gram_length', 'word_start', 'word_end', 'token', 
                        'starting_char_pos']

In [ ]:
textS = pd.Series(proba_df.index.get_level_values('orig_text').unique())
textM = textS.str.count(' ') >= 0
selectedTexts = textS[textM].values # get the original texts

In [ ]:
n_threshold = 2
proba_sub = proba_df[(proba_df.index.get_level_values('n-gram_length') >= n_threshold) &
                     (proba_df.index.get_level_values('orig_text').isin(selectedTexts))]

In [ ]:
max_proba = proba_sub.max(axis=1)
second_proba = proba_sub.quantile(0.999, interpolation='lower', axis = 1)
scoreMargin_m = (max_proba-second_proba) > 0.1  # proba difference between 1st and 2nd must be greater than 0.1
m_val = scoreMargin_m & (~proba_sub.index.get_level_values('token').str.contains('[0-9 ]+ [0-9 ]+'))

In [ ]:
tmpDf = pd.DataFrame({'predicted':proba_sub[m_val].idxmax(axis=1),'score':proba_sub[m_val].max(axis=1)})

In [ ]:
scoreSortedDf = tmpDf[m_val].sort_values(['orig_text','word_start','score'], ascending = False).reset_index()

In [ ]:
v = scoreSortedDf.copy()
scoreSortedDf = scoreSortedDf.assign(OverlapGroup=(len(processed_test)*(v.kthSrs)+ 
                                          (v.word_end - v.word_start.shift(-1)).shift().lt(0).cumsum()))

In [ ]:
hitDf=scoreSortedDf.sort_values(['OverlapGroup','score'],ascending=False).drop_duplicates(['OverlapGroup','predicted']
                                                                                   ).sort_values('orig_text')
hitDf['token_len']=hitDf['token'].str.len()
hitDf['recovered_txt']=hitDf.apply(
    lambda tmpS2:tmpS2.loc['orig_text'][tmpS2.loc['starting_char_pos']:(tmpS2.loc['starting_char_pos']+tmpS2.loc['token_len'])],axis=1)

In [ ]:
hitDf.to_pickle(
    '../results/{myclass}_{model}_prediction.pickle'.format(model = model_iter, myclass = validation_class))